# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686583


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:07<01:43,  3.71s/it]

Rendering models:  10%|█         | 3/30 [00:07<01:12,  2.69s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:47,  1.91s/it]

Rendering models:  20%|██        | 6/30 [00:08<00:33,  1.38s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:11<00:00,  6.16it/s]

Rendering models:  93%|█████████▎| 28/30 [00:11<00:00,  6.35it/s]

Rendering models:  97%|█████████▋| 29/30 [00:11<00:00,  6.20it/s]

Rendering models: 100%|██████████| 30/30 [00:11<00:00,  6.16it/s]

froboy                                0.008440
CThomas                               0.033807
OjO                                   0.001241
kjkavene                              0.000146
not-logged-in-e21b496aa49a15f83c01    0.484777
jmfranci                              0.002474
jfhood                                0.000433
VHOLIK                                0.000223
tosnyder                              0.000145
ncaro                                 0.001012
gavinrider                            0.000171
pangeli5                              0.000430
not-logged-in-3ae935e3afaee843f530    1.129407
Lavadude                              0.003015
not-logged-in-f87600b62b6963cef865    0.101271
jordanrushworth                       0.001327
equidad1                              0.008524
Fla                                   0.004937
Jassels2                              0.034078
ElisabethB                            0.000225
nicholas.                             0.002554
dhaywoo      

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())